In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from SyMBac.external.DeLTA.model import unet_seg
import numpy as np
from PIL import Image
import os
from glob import glob
from tqdm.notebook import tqdm
from csbdeep.utils import normalize
#from stardist import fill_label_holes, random_label_cmap
from skimage.measure import label
#lbl_cmap = random_label_cmap()
from natsort import natsorted

from skimage.filters import threshold_otsu


2023-05-10 03:39:47.397120: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/tmp/ipykernel_2621737/3933000300.py:10: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import distance_transform_edt


In [2]:
data_dirs = glob("data/*")

In [79]:
data_dir = data_dirs[1]
test_data = data_dirs[1]
print(data_dir, test_data)

data/1.5_3 data/1.5_3


In [91]:
for test_data in data_dirs:
    for data_dir in data_dirs:
        print(f"Segmenting {test_data} using {data_dir} model")
        DeLTA_data =  f"{data_dir}/DeLTA"
        training_set = DeLTA_data
        target_size = (832,832)
        input_size = target_size + (1,)
        # Files:

        models = natsorted(glob(DeLTA_data + '/models/*.hdf5'))
        try:
            losses = ([float(x.split("/")[-1].split("_")[-1].split(".hdf5")[0]) for x in models])
        except:
            losses = ([float(x.split("/")[-1].split("-")[-1].split(".hdf5")[0]) for x in models])
        model_file = glob(DeLTA_data + f'/models/*{str(min(losses)).zfill(4)}*.hdf5')[0]
        model = unet_seg(input_size = input_size)
        model.load_weights(model_file)

        test_img_dirs =  natsorted(glob(f"{test_data}/test_data/*[!masks].png"))
        X = [np.pad(np.array(Image.open(x)),16) for x in test_img_dirs]

        n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
        axis_norm = (0,1)   # normalize channels independently
        # axis_norm = (0,1,2) # normalize channels jointly
        all_labels = []
        all_imgs = []
        for x in tqdm(range(len(X))):
            img = normalize(X[x], 1,99.8, axis=(0,1))
            masks = model.predict(img.reshape(1,832,832,1)).reshape(832,832)
            all_labels.append(masks)
            all_imgs.append(img)

        mask = all_labels[-1]

        masks = [label(mask > threshold_otsu(mask)).astype(np.uint16) for mask in all_labels]




        try:
            os.mkdir(f"{test_data}/DeLTA_segmented_by/")
        except:
            pass
        seg_save_dir = f"{test_data}/DeLTA_segmented_by/{data_dir.split('/')[-1]}"
        try:
            os.mkdir(seg_save_dir)
        except:
            pass

        for mask, savename in zip(masks, test_img_dirs):
            filename = savename.split("/")[-1]
            Image.fromarray(mask).save(f"{seg_save_dir}/{filename}")

Segmenting data/5_10 using data/5_10 model


  0%|          | 0/180 [00:00<?, ?it/s]

Segmenting data/5_10 using data/1.5_3 model


  0%|          | 0/180 [00:00<?, ?it/s]

Segmenting data/5_10 using data/2_4 model


...

  0%|          | 0/180 [00:00<?, ?it/s]

Segmenting data/6_12 using data/3_6 model


  0%|          | 0/180 [00:00<?, ?it/s]

Segmenting data/6_12 using data/6_12 model


  0%|          | 0/180 [00:00<?, ?it/s]

'1.5_3'